# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# General libraries
import re
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy import  create_engine
# NLTK libraries
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
# Scikit-learn libraries
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
import pickle

In [2]:
# load data from database
engine = create_engine('sqlite:///etl_pipeline.db')
df = pd.read_sql_table('etl_pipeline', con=engine) 
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df['message'][1]

'Is the Hurricane over or is it not over'

In [4]:
df['related'].unique()

array([1, 0, 2])

In [5]:
df.child_alone.value_counts()

0    26216
Name: child_alone, dtype: int64

In [6]:
# Clean up columns that has value of 2
df = df[df.related != 2]

In [7]:
# Define feature and target variables X and Y
X = df.message.values
Y = np.asarray(df[df.columns[4:]]) # equivalent to Y = df.iloc[:, 4:] 

In [8]:
type(Y)

numpy.ndarray

In [9]:
def tokenize(text):
    #sentence_tokens = sent_tokenize(text)
    word_tokens = word_tokenize(text)
    return word_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
class DenseTransformer(TransformerMixin):

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.todense()

In [11]:
# Building the model
from sklearn.naive_bayes import GaussianNB
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)), 
    ('tfidf', TfidfTransformer()), 
    ('to_dense', DenseTransformer()), 
    ('clf', MultiOutputClassifier(GaussianNB())) 
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [13]:
type(y_train) #.dtype

numpy.ndarray

In [14]:
# # Fit it on a subset of data first
# #'''
# X_train_subset = X_train[0:10]
# y_train_subset = y_train[0:10]
# pipeline.fit(X_train_subset, y_train_subset) 
# #'''

In [15]:
#X_train_subset

In [16]:
#print(type(X_train_subset), type(y_train_subset))
#X_train = X_train.asarray()
#X_train.todense()

In [17]:
# pipeline.fit(X_train, y_train) 

In [18]:
# y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [19]:
#Y

In [29]:
categories = df[df.columns[4:]].columns.values
# print(categories)
#y_test_df = pd.DataFrame(y_test, columns=categories)
#y_pred_df = pd.DataFrame(y_pred, columns=categories)

In [21]:
# for i, c in enumerate(categories):
#     print(c)
#     #print(classification_report(y_test.iloc[:,i] , y_pred[:,i]))
#     print(classification_report(y_test[:,i] , y_pred[:,i]))

In [22]:
#from sklearn.metrics import multilabel_confusion_matrix
#multilabel_confusion_matrix(y_test,y_pred)

### 6. Improve your model
Use grid search to find better parameters. 

In [23]:
# pipeline.get_params()

In [24]:
# parameters = {
#     #'vect__ngram_range':[(1,1),(1,2)],
#     #'vect__binary':[True, False],
#     #'vect__strip_accents':['unicode', None],
#     'tfidf__norm':['l1', 'l2']
# }

# cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=4, verbose=3)

In [25]:
#cv.fit(X_train, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7fce9e442af0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('to_dense',
                                        <__main__.DenseTransformer object at 0x7fcea78d67f0>),
                                       ('clf',
                                        MultiOutputClassifier(estimator=GaussianNB()))]),
             n_jobs=4, param_grid={'tfidf__norm': ['l1', 'l2']}, verbose=3)

In [26]:
#print("\nBest Parameters:", cv.best_params_)


Best Parameters: {'tfidf__norm': 'l1'}


In [27]:
#y_pred = cv.predict(X_test)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
# for i, c in enumerate(categories):
#     print(c)
#     print(classification_report(y_test[:,i] , y_pred[:,i])) #target_names=Y.columns

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [38]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [39]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [40]:
def tokenize(text):
    '''
    This function first normalizes the plain texts, detects URLs in the messages, replace those URLs with a place holder;
    It further tokenizes words and removes stop words;
    After maps and lemmatizes different versions of same words to their root forms, 
    the function finally returns the clean tokens.
    '''
    detected_urls = re.findall(url_regex, text)
    text = text.lower()
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
        
    word_tokens = word_tokenize(text)
    
    words = [w for w in word_tokens if w not in stopwords.words('english')]
    
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in words:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [41]:
# Testing tokenize() function
tokenize('Since then, more than 9 million people have been directly affected by the cyclone which caused extensive damages to houses, water and sanitation infrastructures, harvests, food stocks and livelihoods, also killing livestock and destroying fishing material and other income generating assets.')

['since',
 ',',
 '9',
 'million',
 'people',
 'directly',
 'affected',
 'cyclone',
 'caused',
 'extensive',
 'damage',
 'house',
 ',',
 'water',
 'sanitation',
 'infrastructure',
 ',',
 'harvest',
 ',',
 'food',
 'stock',
 'livelihood',
 ',',
 'also',
 'killing',
 'livestock',
 'destroying',
 'fishing',
 'material',
 'income',
 'generating',
 'asset',
 '.']

In [42]:
class DenseTransformer(TransformerMixin):

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.todense()

In [43]:
# Refitting
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
# More classifiers: AdaBoostClassifier, LinearSVC 
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)), 
    ('tfidf', TfidfTransformer()), 
    ('to_dense', DenseTransformer()), 
    ('clf', MultiOutputClassifier(GaussianNB())) 
])

In [44]:
#pipeline.fit(X_train, y_train) 
#y_pred = pipeline.predict(X_test)

In [45]:
# Grid search
parameters = {
    #'vect__ngram_range':[(1,1),(1,2)],
    'tfidf__norm':['l1', 'l2']
}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=1, verbose=3)

In [46]:
cv.fit(X_train, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV 1/5] END ....................tfidf__norm=l1;, score=0.072 total time= 6.8min
[CV 2/5] END ....................tfidf__norm=l1;, score=0.070 total time= 6.5min
[CV 3/5] END ....................tfidf__norm=l1;, score=0.082 total time= 6.5min
[CV 4/5] END ....................tfidf__norm=l1;, score=0.079 total time= 6.5min
[CV 5/5] END ....................tfidf__norm=l1;, score=0.074 total time= 6.6min
[CV 1/5] END ....................tfidf__norm=l2;, score=0.066 total time= 6.6min
[CV 2/5] END ....................tfidf__norm=l2;, score=0.061 total time= 6.6min
[CV 3/5] END ....................tfidf__norm=l2;, score=0.075 total time= 6.6min
[CV 4/5] END ....................tfidf__norm=l2;, score=0.071 total time= 6.7min
[CV 5/5] END ....................tfidf__norm=l2;, score=0.065 total time= 6.6min


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7fcea7c391f0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('to_dense',
                                        <__main__.DenseTransformer object at 0x7fcea7863df0>),
                                       ('clf',
                                        MultiOutputClassifier(estimator=GaussianNB()))]),
             n_jobs=1, param_grid={'tfidf__norm': ['l1', 'l2']}, verbose=3)

In [47]:
type(X_train)

numpy.ndarray

In [48]:
print("\nBest Parameters:", cv.best_params_)


Best Parameters: {'tfidf__norm': 'l1'}


In [49]:
y_pred = cv.predict(X_test)

In [51]:
# Reevaluation
for i, c in enumerate(categories):
    print(c)
    print(classification_report(y_test[:,i] , y_pred[:,i])) #target_names=Y.columns

related
              precision    recall  f1-score   support

           0       0.32      0.63      0.43      1537
           1       0.84      0.59      0.70      4970

    accuracy                           0.60      6507
   macro avg       0.58      0.61      0.56      6507
weighted avg       0.72      0.60      0.63      6507

request
              precision    recall  f1-score   support

           0       0.92      0.73      0.81      5436
           1       0.33      0.68      0.45      1071

    accuracy                           0.72      6507
   macro avg       0.63      0.71      0.63      6507
weighted avg       0.82      0.72      0.75      6507

offer
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      6479
           1       0.02      0.07      0.03        28

    accuracy                           0.98      6507
   macro avg       0.51      0.53      0.51      6507
weighted avg       0.99      0.98      0.99      650

In [53]:
#print(classification_report(y_test, y_pred, target_names=category_names))
#results = pd.DataFrame(columns=['Category', 'f_score', 'precision', 'recall'])

In [54]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
accuracy_score(y_test, y_pred, normalize=True)

0.07161518364837867

In [55]:
f1_score(y_test, y_pred, average='weighted')

/Users/lin/miniconda3/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1495: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


0.4025353094765057

In [56]:
f1_score(y_test, y_pred, average='macro')

/Users/lin/miniconda3/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1495: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


0.1563271449686524

In [57]:
f1_score(y_test, y_pred, average='micro')

0.3095840559520086

In [58]:
recall_score(y_test, y_pred, average='micro')

0.5369858960449748

In [59]:
precision_score(y_test, y_pred, average='macro')

/Users/lin/miniconda3/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.11675232192272704

### 9. Export your model as a pickle file

In [63]:
# Save the model to disk
pickle.dump(cv.best_estimator_, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

#### Imbalanced Data
It can be seen that the count for different categories are imbalanced, this will normally require us to look carefully at any sources of bias during data collections are accounted for. 